# KERAS-TF2.0-FaceAttribute

## 1. Dataset

In [12]:
import tensorflow as tf
import numpy as np
import random
import os

### Load labels

In [13]:
def load_labels(label_file):
    f = open(label_file)
    # line 1: number of images
    num_imgs = int(f.readline())
    # line 2: attribute names, 40 in total
    attr_names = f.readline().split()
    # line 3 to end: 00xx.jpg -1 1 -1 1 ...
    labels = []
    for i in range(num_imgs):
        labels.append(list(map(np.float32, f.readline().split()[1:])))
    labels = np.array(labels)
    labels[labels<0] = 0
    return labels

labels = load_labels('../dataset/list_attr_celeba.txt')

### Load images

In [14]:
image_dir = '../dataset/img_align_celeba'
image_paths = os.listdir(image_dir)
image_paths.sort()
for i in range(len(image_paths)):
    image_paths[i] = os.path.join(image_dir,image_paths[i])
image_count = len(image_paths)

### Build dataset

In [15]:
def load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    # uint8 range: [0,255]
    img = tf.image.resize(img, [192, 160])
    # new range: [-128,127]
    img = img-128
    return img, label

TRAIN_SIZE = int(0.7*image_count)
VAL_SIZE   = int(0.15*image_count)
BATCH_SIZE = 32

ds_train = tf.data.Dataset.from_tensor_slices((image_paths[:TRAIN_SIZE], labels[:TRAIN_SIZE])).map(load_and_preprocess)
ds_train = ds_train.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=8192))
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

ds_val  = tf.data.Dataset.from_tensor_slices((image_paths[TRAIN_SIZE:TRAIN_SIZE+VAL_SIZE], labels[TRAIN_SIZE:TRAIN_SIZE+VAL_SIZE])).map(load_and_preprocess)
ds_val  = ds_val.batch(BATCH_SIZE)

ds_test = tf.data.Dataset.from_tensor_slices((image_paths[TRAIN_SIZE+VAL_SIZE:], labels[TRAIN_SIZE+VAL_SIZE:])).map(load_and_preprocess)
ds_test = ds_test.batch(BATCH_SIZE)

## 2. Model

In [26]:
mnet = tf.keras.applications.mobilenet.MobileNet(input_shape=(192,160,3),alpha=0.5,include_top=False,weights=None,pooling='avg')
mnet = tf.keras.Sequential([mnet,tf.keras.layers.Dense(labels.shape[1],activation='sigmoid',name='top_dense')], name='mnet_050_faceattr')
mnet.summary()

if os.path.exists('mnet.best.hdf5'):
    mnet.load_weights('mnet.best.hdf5')

mnet.compile(optimizer=tf.keras.optimizers.RMSprop(),
            loss='binary_crossentropy',
            metrics=['binary_accuracy'])

Model: "mnet_050_faceattr"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_0.50_192 (Model)   (None, 512)               829536    
_________________________________________________________________
top_dense (Dense)            (None, 40)                20520     
Total params: 850,056
Trainable params: 839,112
Non-trainable params: 10,944
_________________________________________________________________


## 3. Train

In [25]:
model_saver = tf.keras.callbacks.ModelCheckpoint(filepath='mnet.best.hdf5', period=10, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
steps_per_epoch=tf.math.ceil(TRAIN_SIZE/BATCH_SIZE).numpy()
mnet.fit(ds_train, validation_data=ds_val, epochs=100, steps_per_epoch=2, validation_steps=2, verbose=2, callbacks=[model_saver])


Epoch 1/100


ValueError: Error when checking input: expected mobilenet_0.50_160_input to have shape (160, 160, 3) but got array with shape (192, 160, 3)